In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=a9712f0b5b75cbffcb09ba54d65c7ac0e2e1a140ca404e9d5ef85ff16d2ad3a1
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F10 '''
def F10(X):
    f = bn.F10()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 692.83 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_10_2000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_10_400Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 419058491848833.94, tolerance: 322810890172622.06
  positive)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,101):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 100, [5] * 100 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 100000)

n_obs , dim =  30, 100
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,300])
for i in range(G[1].shape[0]):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=935657, Tue Apr 14 11:43:51 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 7.146527049423970e+07 1.0e+00 2.40e+00  2e+00  2e+00 0:00.0
    2     34 5.297690770457961e+07 1.0e+00 2.33e+00  2e+00  2e+00 0:00.1
    3     51 4.808813775999217e+07 1.0e+00 2.27e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [80628737.5901297] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [80209924.19366257] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   88   1496 -1.534620474273728e+08 1.3e+00 9.10e-01  9e-01  9e-01 0:03.1
  100   1700 -1.737033442897279e+08 1.3e+00 7.29e-01  7e-01  7e-01 0:03.6
  200   3400 -2.198364913913904e+08 1.3e+00 1.21e-01  1e-01  1e-01 0:07.1
  300   5100 -2.214966877281793e+08 1.3e+00 1.90e-02  2e-02  2e-02 0:12.9
  400   6800 -2.215328086346055e+08 1.3e+00 3.41e-03  3e-03  3e-03 0:19.5
  500   8500 -2.215336415179777e+08 1.4e+00 5.74e-04  5e-04  6e-04 0:23.0
  600  10200 -2.215336773077119e+08 1.4e+00 9.15e-05  9e-05  9e-05 0:26.5
  700  11900 -2.215336780546494e+08 1.4e+00 1.56e-05  1e-05  2e-05 0:30.1
  800  13600 -2.215336780787578e+08 1.4e+00 2.81e-06  3e-06  3e-06 0:33.7
  900  15300 -2.215336780793661e+08 1.4e+00 4.72e-07  4e-07  5e-07 0:37.3
 1000  17000 -2.215336780793866e+08 1.4e+00 8.29e-08  7e-08  8e-08 0:40.9
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -2.215336780793870e+08 1.4e+00 3.18e-08  3e-08  3e-08 0:44.5
 1200  20400 -2.215336780793871e+08 1.5e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [96867203.60768202] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=96867203.607682, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=96867203.607682, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iterati

    2     34 3.805429786352285e+07 1.0e+00 2.33e+00  2e+00  2e+00 0:00.1
    3     51 1.880590960995422e+07 1.0e+00 2.28e+00  2e+00  2e+00 0:00.1


...

   88   1496 -9.819718833198266e+07 1.3e+00 1.45e+00  1e+00  1e+00 0:03.1
  100   1700 -1.169594767804556e+08 1.3e+00 1.23e+00  1e+00  1e+00 0:03.5
  200   3400 -2.177180433121577e+08 1.4e+00 2.02e-01  2e-01  2e-01 0:07.0
  300   5100 -2.214476351019234e+08 1.4e+00 3.18e-02  3e-02  3e-02 0:10.4
  400   6800 -2.215317887584641e+08 1.4e+00 4.83e-03  5e-03  5e-03 0:13.9
  500   8500 -2.215336102820997e+08 1.4e+00 8.11e-04  8e-04  8e-04 0:17.3
  600  10200 -2.215336763119935e+08 1.4e+00 1.42e-04  1e-04  1e-04 0:20.9
  700  11900 -2.215336780290720e+08 1.4e+00 2.17e-05  2e-05  2e-05 0:24.4
  800  13600 -2.215336780781021e+08 1.4e+00 4.23e-06  4e-06  4e-06 0:27.9
  900  15300 -2.215336780793480e+08 1.4e+00 6.85e-07  6e-07  7e-07 0:31.3
 1000  17000 -2.215336780793863e+08 1.4e+00 1.06e-07  1e-07  1e-07 0:34.8
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -2.215336780793872e+08 1.4e+00 3.49e-08  3e-08  3e-08 0:38.3
 1200  20400 -2.215336780793871e+08 1.5e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [96867203.60768202] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=96867203.607682, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=96867203.607682, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iterati

    2     34 6.484502048657431e+07 1.0e+00 2.33e+00  2e+00  2e+00 0:00.1
    3     51 3.205576140229264e+07 1.0e+00 2.28e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1.08543139e+08] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   76   1292 -1.656754783172199e+08 1.3e+00 1.48e+00  1e+00  2e+00 0:03.2
  100   1700 -1.949231670734745e+08 1.3e+00 1.00e+00  1e+00  1e+00 0:04.2
  200   3400 -3.104932014692992e+08 1.4e+00 3.67e-01  3e-01  4e-01 0:08.3
  300   5100 -3.318613850996515e+08 1.4e+00 1.01e-01  9e-02  1e-01 0:12.5
  400   6800 -3.359471626356995e+08 1.6e+00 5.36e-02  5e-02  6e-02 0:16.6
  500   8500 -3.421046886241671e+08 2.7e+00 1.09e-01  1e-01  2e-01 0:20.8
  600  10200 -3.458671437216396e+08 3.1e+00 4.60e-02  4e-02  9e-02 0:25.0
  700  11900 -3.474713029398040e+08 3.1e+00 4.46e-02  4e-02  8e-02 0:29.3
  800  13600 -3.494434038899327e+08 3.8e+00 5.04e-02  4e-02  9e-02 0:33.4
  900  15300 -3.516354542624475e+08 4.6e+00 5.29e-02  4e-02  1e-01 0:37.6
 1000  17000 -3.536141918230405e+08 4.9e+00 4.71e-02  4e-02  1e-01 0:41.7
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -3.541988776000240e+08 5.0e+00 1.92e-02  2e-02  5e-02 0:45.9
 1200  20400 -3.544252993107154e+08 5.0e

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F10_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F10(X_Values.iloc[i,:100])
    SVM_Fun [i] = F10(X_Values.iloc[i,100:200])
    ELN_Fun [i] = F10(X_Values.iloc[i,200:300])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([52325463.78352132]), count=array([1])) 0.6683075869612061
SVM
ModeResult(mode=array([62357239.74004319]), count=array([1])) 43439188.69853606
ELN
ModeResult(mode=array([52567882.18125664]), count=array([1])) 14204036.896250622
